# T5.1 Panel de control con PowerBI
## Ralphy Núñez Mercado
Hai que facer un proxecto con PowerBI que teña os seguintes elementosPower BI:

Dous informes (e vista para móbiles e ordenador por cada informe).
Alomenos 2 gráficas ou elementos gráficos ordinarios de PowerBI.
1 Mapa (aínda que sexa con outros datos sen relación).
Dúas orixes de datos, por exemplo un excel e un JSON.
Relacións entre os datos (alomenos algunha que teña sentido).
Un obxecto visual de Python.
Un orixe de datos cun script de Python.
Emprega varios orixes de datos (diferentes) para comparar.

Na práctica debe verse algo de:

Scrapping
PANDAS
Spark-HDFS <-> PowerBi

E ademáis que quede bonito o deseño do informe :)

Deberase facer con PowerBI Desktop e publicarase a app.powerbi.com coa conta @fernandowirtz.com.
Subirase a esta tarefa o arquivo .pbix. Se é moi grande para subir, darase unha ligazón ao arquivo en OneDrive (coa conta @fernandowirtz.com).
Compartirase a ligazón ao informe publicado en app.powerbi.com tamén co profe (pode poñerse no texto da tarefa ou engadir ao membro da organización co email: scj@fernandowirtz.com)

### ⬇️ Instalar las librerías

In [ ]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true
!pip install pyodbc -y || true

^C


### 🦎 Instalar gecko Driver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\ralphy.nunezmercado\\.wdm\\drivers\\geckodriver\\win64\\v0.36.0\\geckodriver.exe'

### ⬇️ Importar las librerías

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import random

### Conectar el driver de Firefox

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import random

options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
options.set_preference("general.useragent.override", 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0")

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

# Acepta cookies si aparece el botón
try:
    btn_cookies = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar') or contains(., 'Aceptar todas') or contains(., 'Aceptar todo')]"))
    )
    btn_cookies.click()
    print("Cookies aceptadas")
    time.sleep(1)
except Exception:
    print("No apareció banner de cookies o ya estaba aceptado")

datos_coches = []
pagina = 1

while True:
    print(f"Extrayendo página {pagina}...")
    # Espera a que carguen los artículos de coches
    try:
        wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'article')))
    except Exception:
        print("No se encontraron coches en la página, saliendo del bucle.")
        break

    # Vuelve a buscar los artículos en cada iteración
    main = driver.find_element(By.TAG_NAME, 'main')
    articulos = main.find_elements(By.TAG_NAME, 'article')

    for idx in range(len(articulos)):
        try:
            # Vuelve a buscar el artículo por índice para evitar stale element
            main = driver.find_element(By.TAG_NAME, 'main')
            articulos_actualizados = main.find_elements(By.TAG_NAME, 'article')
            articulo = articulos_actualizados[idx]
            
            marca = articulo.get_attribute('data-make') or ''
            modelo = articulo.get_attribute('data-model') or ''
            precio = articulo.get_attribute('data-price') or ''
            if not precio:
                try:
                    precio = articulo.find_element(By.CSS_SELECTOR, '[data-testid="regular-price"]').text
                    precio = re.sub(r'[^\d]', '', precio)
                except:
                    precio = ''
            try:
                km_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-mileage_road"]').text
                km = re.sub(r'[^\d]', '', km_text)
            except:
                km = ''
            try:
                year_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-calendar"]').text
                year = year_text.split('/')[-1] if '/' in year_text else year_text
            except:
                year = ''
            try:
                combustible = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-gas_pump"]').text
            except:
                combustible = ''
            try:
                cv_text = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-speedometer"]').text
                match = re.search(r'\((\d+)\s*CV\)', cv_text)
                cv = match.group(1) if match else ''
            except:
                cv = ''
            try:
                ubicacion_raw = articulo.find_element(By.CSS_SELECTOR, '[data-testid="sellerinfo-address"]').text
                if "Contáctanos en:" in ubicacion_raw:
                    ubicacion = ubicacion_raw.split()[-1]
                else:
                    ubicacion = ubicacion_raw
            except:
                ubicacion = ''
            try:
                transmision = articulo.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-transmission"]').text
            except:
                transmision = ''
            datos_coches.append([
                marca.capitalize(), modelo, precio, km, year, combustible, cv, ubicacion, transmision
            ])
        except Exception as e:
            print(f"Error extrayendo un coche: {e}")
            continue

    # Espera aleatoria para parecer humano
    time.sleep(random.uniform(2, 4))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1, 2))

    # Busca el botón "Siguiente" en la paginación
    try:
        paginacion = driver.find_element(By.CSS_SELECTOR, '[data-testid="listpage-pagination"]')
        next_btn_li = paginacion.find_element(By.XPATH, ".//li[contains(@class, 'prev-next') and not(contains(@class, 'previous'))]")
        next_btn = next_btn_li.find_element(By.TAG_NAME, "button")
        if next_btn.get_attribute("aria-disabled") == "true":
            print("No hay más páginas.")
            break
        next_btn.click()
        pagina += 1
    except Exception as e:
        print("No hay más páginas o no se encontró el botón de siguiente.")
        break

# Guardar en CSV
with open('coches.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'Marca', 'Modelo', 'Precio', 'KM', 'año',
        'Tipo de combustile', 'CV', 'Ubicacion', 'Trasmision'
    ])
    writer.writerows(datos_coches)

print(f"¡Datos guardados en coches.csv! Total coches: {len(datos_coches)}")
driver.quit()


Cookies aceptadas
Extrayendo página 1...
Extrayendo página 2...
Extrayendo página 3...
Extrayendo página 4...
Error extrayendo un coche: Message: The element with the reference 46070174-7fcf-4228-8453-d0c7060d555d is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:796:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:405:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:263:20
cloneObject/result<@chrome://remote/content/marionette/json.sys.mjs:53:52
cloneObject@chrome://remote/content/marionette/json.sys.mjs:53:25
de